<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Youtube_NLP_Text_Representaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# !pip install sklearn -q
# !pip install spacy -q
# !python -m spacy download en_core_web_sm

In [34]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

import string
import spacy
np.random.seed(42)

In [35]:
data = pd.read_csv("Depressed1.csv", engine="python",encoding='ISO-8859-1')
data.head()

,clean_text,is_depression
0,Abnormal\nI'm so used to being depressed and a...,1
1,Does therapy help???\nFor 5 years I have had t...,1
2,"She said yes!\nI did it, I asked her to be my ...",1
3,The next 10 years will decide if I live the re...,1
4,struggling to plug back in\nthrowaway. anyway....,1


In [36]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'me', 'take', 'own', 'whatever', 'name', 'whereby', 'two', 'often', 'eleven', 'sometimes', 'wherein', 'hereby', 'many', 'above', 'fifteen', 'by', 'thereupon', 'per', '’re', 'a', 'somehow', 'upon', 'us', "'re", 'nobody', 'behind', "'ve", '’d', 'still', 'will', 'top', 'be', 'for', 'themselves', 'thus', 'fifty', 'part', 'six', 'rather', 'were', 'thru', '‘re', 'further', 'himself', 'hers', 'would', 'noone', 'another', 'around', 'that', 'forty', 'up', 'whenever', 'does', 'otherwise', 'myself', 'together', 'their', 'even', 'since', 'below', 'thereby', 'without', 'via', 'therein', '’ve', '‘ve', 'everywhere', 'nor', 'this', 'four', 'besides', 'had', 'hundred', 'five', 'why', 'everything', 'yet', 'because', 'do', 'nowhere', '‘d', 'she', 'did', 'has', 'her', 'almost', 'throughout', 'our', 'however', 'anywhere', 'most', 'again', 'same', 'third', 'also', 'something', 'go', 'doing', 'very', 'whereas', 'have', 'became', 'other', 'serious', 'become', 'yourself', 'get', 'herein', 'where', 'done', 'sh

In [37]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [38]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [39]:
sentence = "I am eating apple ?"
spacy_tokenizer(sentence)

['eat', 'apple']

In [40]:
count_vector = CountVectorizer(tokenizer = spacy_tokenizer)
# tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [41]:
count_vector.fit_transform(["I am eating apple, I like apple","I am playing cricket"]).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[2, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]])

In [42]:
count_vector.get_feature_names_out()

array(['apple', 'cricket', 'eat', 'like', 'play'], dtype=object)

In [43]:
count_vector.vocabulary_

{'eat': 2, 'apple': 0, 'like': 3, 'play': 4, 'cricket': 1}

In [46]:
from sklearn.model_selection import train_test_split

X = data['clean_text'] # the features we want to analyze
ylabels = data['is_depression'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,stratify=ylabels)

In [47]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [48]:
X_train_vetcors= count_vector.fit_transform(X_train)
X_test_vetcors= count_vector.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [49]:
X_train_vetcors.shape

(1470, 9479)

In [50]:
X_test_vetcors.shape

(368, 9479)

In [51]:
X_train_vetcors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [52]:
classifier.fit(X_train_vetcors,y_train)

LogisticRegression()

In [53]:
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.8804347826086957
Logistic Regression Precision: 0.8872727272727273
Logistic Regression Recall: 0.9494163424124513


In [54]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
X_train_vetcors= tfidf_vector.fit_transform(X_train)
X_test_vetcors= tfidf_vector.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [55]:
classifier = LogisticRegression()
classifier.fit(X_train_vetcors,y_train)
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.8641304347826086
Logistic Regression Precision: 0.8461538461538461
Logistic Regression Recall: 0.9844357976653697
